Creates:
* xray_comp.pdf

In [74]:
from astropy.table import Table
from astropy.io import ascii as asc
import astropy.units as u
from astropy.time import Time
from matplotlib import pyplot as plt
import numpy as np
%matplotlib 
import matplotlib as mpl
from utilities_az import supernova

Using matplotlib backend: Qt5Agg


In [4]:
plt.style.use(['seaborn-paper', 'az-paper-onecol'])

# Read in SNaX data

In [5]:
#        0       1           2                  3             4              5          6        7           8                                  9              10         11                           12           13           14          15                       16                              17                          18        19          
names=["name","type","Date Exploded (JD)","Coordinates","Distance (Mpc)","galaxy","redshift","redshiftErr","Observation Start Date (JD)","instrument","Age (days)","Flux (10^-13 erg cm^-2 s^-1)","isUpperBound","fluxErrL","fluxErrH","Energy Lower Bound (KeV)","Energy Upper Bound (KeV)","Luminosity (10^39 erg s^-1)","lumErrL","lumErrH","model","dateExplodedRef","redshiftRef","dateExplodedRef","coordsRef","distRef","dateObservedRef","fluxRef", "junk"]
name = []
sntype = []
age = []# (days)
isUpperBound = []
luminosity = [] # (10^39 erg s^-1) 
lumErrL =[]
lumErrH =[]

ofile = open('../../data/xray/SNaX.TSV', 'r')
for iline in ofile:
    if iline.startswith('SN'):
        sline = iline.split('\t')
        name.append(sline[0])
        sntype.append(sline[1])
        age.append(float(sline[10]))
        isUpperBound.append(bool(int(sline[12])))
        luminosity.append(float(sline[17]))
        lumErrL.append(float(sline[18]))
        lumErrH.append(float(sline[19]))
tbdata = Table([name, sntype, age, isUpperBound, luminosity, lumErrL, lumErrH], 
              names=['name', 'sntype', 'age', 'isUpperBound', 'luminosity', 'lumErrL', 'lumErrH'])

# Read in data from Sam

In [6]:
ofile_obs = asc.read('../../data/xray/asassn15oz_SpectrumData.csv', data_start=2, delimiter=',',
                    names=['ObsID','SourceCount','SourceRate','BGCount','BGRate','BackedSpectrum',
                           'BackedSpectrumError','Flux','FluxError','Luminosity','LuminosityError','PercentTotal',
                           'ExposureTime','MJD','Date','Probability','Confidence'])

# Read in data from Stefano

In [143]:
ofile_ximage = asc.read('../../data/xray/upper_lim_table.txt', data_start=1,
                       names=['visit','date-obs','exptime','obsid','cts/s','Flux','unabsorbed-flux','NH'],
                       format='fixed_width')
distance = (28.83*u.Mpc).to(u.cm)
ofile_ximage['luminosity'] = (4*np.pi*distance**2)*ofile_ximage['unabsorbed-flux']
ofile_ximage['luminosity'][ofile_ximage['luminosity']==0]=np.nan
ofile_ximage['MJD'] = Time(ofile_ximage['date-obs']).mjd

In [141]:
np.array(ofile_ximage['luminosity'])/10**39

array([0.0, 24.0567030606753, 100.34399912451995, 0.0, 0.0,
       66.17333698459423, 37.91988787530174, 0.0, 24.016923477276084, 0.0,
       22.34618097450905, 0.0, 13.535003251582918, 0.0, 0.0,
       30.74961796759323, 0.0, 50.19188935395958, 56.50689821858497, 0.0,
       0.0, 34.30989068182298, 18.885357218777344, 0.0, 0.0, 0.0,
       14.31070512786761, 0.0, 0.0, 12.679742208499796, 0.0,
       7.992712794487283, 0.0, 13.385829813835862, 0.0,
       138.63184814626442, 0.0, 0.0, 0.0, 39.77958339921504,
       108.59826267985707, 0.0, 0.0, 0.0, 19.740618261860465, 0.0, 0.0,
       132.36656376088808, 0.0, 13.64439710593076], dtype=object)

In [129]:
sn15oz = supernova.LightCurve2('asassn-15oz')
sn15oz_phase = Time(['2015-09-05', '2015-11-18']) - Time(sn15oz.jdexpl, format='jd')
#sn15oz_phase = Time(['2015-09-05', '2015-11-18']) - Time('2015-08-27')

In [163]:
fig = plt.figure()
fig.subplotpars.update(left=0.25)
ax = fig.add_subplot(1,1,1)

#l1 = ax.fill_between(phase, Lx1_low, Lx1_up, alpha=0.3, label=r'$10^{-7} M_{\odot}/yr$')
#l2 = ax.fill_between(phase, Lx2_low, Lx2_up, alpha=0.3, label=r'$10^{-6} M_{\odot}/yr$')
#l3 = ax.fill_between(phase, Lx3_low, Lx3_up, alpha=0.3, label=r'$10^{-5} M_{\odot}/yr$')

leg_lines, leg_labels = ax.get_legend_handles_labels()

snnames = set(tbdata['name'])
for snname in snnames:
    indx = (tbdata['name']==snname) & (tbdata['age']<100)
    if (indx==True).any():
        sntype = tbdata[indx]['sntype'][0]

        if 'L' in sntype:
            iline = ax.errorbar(tbdata[indx]['age'], tbdata[indx]['luminosity'], fmt='s',
                         yerr=[tbdata[indx]['lumErrH'], tbdata[indx]['lumErrL']], ls='--')
            leg_lines.append(iline[0])
            leg_labels.append('{}, {}'.format(snname, tbdata[indx]['sntype'][0]))
            
            ax.errorbar(tbdata[indx]['age'][tbdata[indx]['isUpperBound']], tbdata[indx]['luminosity'][tbdata[indx]['isUpperBound']], fmt='s',
                         yerr=[0.25]*len(tbdata[indx]['age'][tbdata[indx]['isUpperBound']]),
                         uplims = [True]*len(tbdata[indx]['age'][tbdata[indx]['isUpperBound']]), ecolor=iline[0].get_color())
            
            
            
        elif 'n' not in sntype:
            iline = ax.errorbar(tbdata[indx]['age'], tbdata[indx]['luminosity'], fmt='d',
                         yerr=[tbdata[indx]['lumErrH'], tbdata[indx]['lumErrL']], ls=':')
            leg_lines.append(iline[0])
            leg_labels.append('{}, {}'.format(snname, tbdata[indx]['sntype'][0]))
            ax.errorbar(tbdata[indx]['age'][tbdata[indx]['isUpperBound']], tbdata[indx]['luminosity'][tbdata[indx]['isUpperBound']], fmt='d',
                         yerr=[0.25]*len(tbdata[indx]['age'][tbdata[indx]['isUpperBound']]),
                         uplims = [True]*len(tbdata[indx]['age'][tbdata[indx]['isUpperBound']]), ecolor=iline[0].get_color())
ax.set_yscale("log")
iline = ax.errorbar(ofile_ximage['MJD']-Time(sn15oz.jdexpl, format='jd').mjd, np.array(ofile_ximage['luminosity'])/10**39, np.array(ofile_ximage['luminosity'])/10**39*0.2, 
                    uplims=True, fmt='.')
leg_lines.append(iline[0])
leg_labels.append('ASASSN-15oz, IIL')
ax.set_xlim(0,160)
#ax.set_ylim(-10, 20)
ax.set_xlabel('Phase (day)')
ax.set_ylabel(r'Luminosity (x10$^{39}$ erg $\rm s^{-1}$)', position=(1,0.38))
ax.legend(leg_lines, leg_labels, bbox_to_anchor=[0.55, 0.65, 0.4, 0.4], framealpha=1.0)
plt.savefig('xray_comp.pdf')

# Calculate the expected luminosity from the forward shock

In [111]:
#From Dwarkadas 2014
def calc_xray_luminosity(Mdot, vwind, phase):
    gff=1
    Cn = 1 #for CSM forward shock
    Mdot = Mdot/(10**-5)
    vwind = vwind/10
    phase = phase/10
    Lx = 3.0E39*gff*Cn * (Mdot/vwind)**2 * 1./(phase)
    Lx = Lx/10**39
    Lx = Lx.astype(np.float)
    return Lx



In [112]:
phase = np.arange(1, 115)
vwind1=10 #km/s
vwind2= 100 #km/s
Mdot1 = 10**-7
Mdot2 = 10**-6
Mdot3 = 10**-5

Lx1_up = calc_xray_luminosity(Mdot1, vwind1, phase)
Lx1_low = calc_xray_luminosity(Mdot1, vwind2, phase)

Lx2_up = calc_xray_luminosity(Mdot2, vwind1, phase)
Lx2_low = calc_xray_luminosity(Mdot2, vwind2, phase)

Lx3_up = calc_xray_luminosity(Mdot3, vwind1, phase)
Lx3_low = calc_xray_luminosity(Mdot3, vwind2, phase)

In [50]:
ofile_ximage

directory,exptime,cts/s,FLux,unabsorbed-flux,NH
int64,float64,float64,float64,float64,float64
34040001,4415.335,0.00401,1.386e-13,1.444e-13,1.18e+20
34040002,4352.673,0.0042,1.452e-13,1.513e-13,1.18e+20
34040004,526.533,0.0215,7.431e-13,7.745e-13,1.18e+20
34040005,3046.37,0.0036,1.244e-13,1.297e-13,1.18e+20


In [51]:
ofile_obs['ObsID'] == ofile_ximage['directory'][1]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [98]:
ofile_ximage['MJD']-Time(sn15oz.jdexpl, format='jd').mjd

8.743779299999005
8.5
13.65492809999705
13.657221900000877


In [161]:
print(ofile_ximage['luminosity'])

      luminosity      
         cm2          
----------------------
                   nan
2.4056703060675297e+40
1.0034399912451994e+41
                   nan
                   nan
 6.617333698459422e+40
 3.791988787530174e+40
                   nan
 2.401692347727608e+40
                   nan
                   ...
1.0859826267985706e+41
                   nan
                   nan
                   nan
1.9740618261860465e+40
                   nan
                   nan
1.3236656376088806e+41
                   nan
 1.364439710593076e+40
Length = 50 rows
